In [1]:
import os 
import re 
import pandas as pd
from tqdm import tqdm
from collections import Counter
from utils import get_completion, num_tokens_from_string, compute_token_price
import pickle

In [2]:
with open("../qa_results/all_qa_dfs.pkl", "rb") as infile:
    dfs = pickle.load(infile)

In [3]:
count = 0
for e,i in enumerate(dfs):
    for p in dfs[e]["Query"].tolist():
        if re.search(r" secondo il testo\?", p):
            count+=1
print(f"{count} queries contains ' secondo il testo?' to be removed")

315 queries contains ' secondo il testo?' to be removed


In [ ]:
#get rid of "secondo il testo?"
# for e,i in enumerate(dfs):
#     dfs[e]["Query"] = dfs[e]["Query"].apply(lambda x: re.sub(r" secondo il testo\?", r"\?", x)) 

In [5]:
evaluation_prompt = open("../prompts/eval_prompt.txt", "r").read()
print(evaluation_prompt)

###Instruction###
Sei un esperto in giurisprudenza. Di seguito ti verranno mostrati un testoe una domanda.
Il tuo compito è stabilire se la risposta alla domanda è contenuta nel testo.
Puoi utilizzare solo i seguenti due OUTPUT validi: ["SI", "NO"].
L'OUTPUT è "SI" se la la risposta alla domanda è contenuta nel testo.
L'OUTPUT "NO" se la risposta alla domanda non è contenuta nel testo.
Per poter dire "SI" la risposta alla domanda deve essere strettamente e chiaramente nel testo.
Restituisci solamente "SI" o "NO" e null'altro.


###TESTO###
{text}

###DOMANDA###
{query}

OUTPUT: 


In [7]:
eval_dfs = []
for n,book in enumerate(dfs, 1):
    print(f"Processing qas of book_{n}")
    answ = []
    for row in tqdm(book.itertuples()):
        query = re.sub(" secondo il testo?", "", row.Query[2:]).strip()
        passage = row.Passage.strip()
        prompt =  re.sub(r"\{query\}", query, re.sub(r"\{text\}", passage, evaluation_prompt))
        answ.append(re.sub(r"\"", "", get_completion(prompt, "gpt-4o")).strip())
        
    
    print(f"Results for book {n}: {Counter(answ)}\n")
    
    #Build and save a df for each book
    book["IsQueryRelevant"] = answ
    eval_dfs.append(book)
    book.to_csv(f"../automatic_eval_results/autoeval_qabook_{n}.tsv", sep = "\t")
    print(f"Saved df for book{n}")


    

Processing qas of book_1


1115it [11:29,  1.62it/s]


Results for book 1: Counter({'SI': 736, 'NO': 376, 'Dal giorno del suo ritorno nel luogo in cui è nato il figlio o in cui è la residenza familiare se egli ne era lontano.': 1, 'Il figlio o il genitore o convivente che maltratta o abusa del minore possono essere allontanati dalla residenza familiare.': 1, 'uno dei genitori, se idoneo all’ufficio, o in mancanza, altra persona': 1})

Saved df for book1
Processing qas of book_2


874it [08:50,  1.65it/s]


Results for book 2: Counter({'SI': 606, 'NO': 265, 'Chiunque vi ha interesse': 1, 'Chiunque crede di avervi interesse': 1, 'Il coerede che conferisce un immobile in natura può ritenerne il possesso sino all’effettivo rimborso delle somme che gli sono dovute per spese e miglioramenti.\n\nSI': 1})

Saved df for book2
Processing qas of book_3


...

Results for book 4: Counter({'SI': 1414, 'NO': 698, 'Un’accettazione non conforme alla proposta equivale a nuova proposta. \n\nQuindi, la risposta alla domanda è contenuta nel testo.\n\nSI': 1, 'Il termine di preavviso può essere sostituito dal pagamento di una corrispondente indennità.': 1, 'Il sequestratario non può consentire locazioni per durata superiore a quella stabilita per le locazioni a tempo indeterminato.': 1, 'l’equivalente, oppure la restituzione e un’indennità per la diminuzione di valore': 1})

Saved df for book4
Processing qas of book_5


2132it [22:16,  1.60it/s]


Results for book 5: Counter({'SI': 1464, 'NO': 664, 'Il rischio delle cose conferite in godimento resta a carico del socio che le ha conferite.': 1, 'Presso l’ufficio del registro in cui è iscritta la sede secondaria deve essere altresì depositata la firma autografa del rappresentante preposto all’esercizio della sede medesima. \n\nQuindi, la risposta è: \n\nSI': 1, 'Al momento della sottoscrizione.': 1, 'partecipazioni in imprese controllate; partecipazioni in imprese collegate; partecipazioni in imprese controllanti; altre partecipazioni': 1})

Saved df for book5
Processing qas of book_6


890it [10:45,  1.38it/s]

Results for book 6: Counter({'SI': 538, 'NO': 351, 'nel domicilio eletto': 1})

Saved df for book6


In [ ]:
# #save a binary file for all evaluated dfs
# with open("../automatic_eval_results/all_autoeval_dfs.pkl", "wb") as outfile:
#     pickle.dump(eval_dfs, outfile)

In [40]:

for n,d in enumerate(dfs,1):
    io_ratio = d["Articolo di riferimento"].shape[0] / d["Articolo di riferimento"].unique().shape[0] 
    co_queries = d[d["IsQueryRelevant"] == "SI"].shape[0]/d.shape[0]
    failed_answ = len([i for i in d["IsQueryRelevant"].tolist() if i not in ["SI", "NO"]])
    
    print(f"Stat for book {n}")
    print(f"The ratio betweween generated outputs and inputs is {round(io_ratio, 3)}")
    print(f"Queries judged as coherent {round(co_queries,3)*100}%")
    print(f"Number of failed answers: {failed_answ}\n\n-----------------------------------------------------")


Stat for book 1
The ratio betweween generated outputs and inputs is 2.844
Queries judged as coherent 66.0%
Number of failed answers: 3

-----------------------------------------------------
Stat for book 2
The ratio betweween generated outputs and inputs is 2.533
Queries judged as coherent 69.3%
Number of failed answers: 3

-----------------------------------------------------
Stat for book 3
The ratio betweween generated outputs and inputs is 2.643
Queries judged as coherent 65.4%
Number of failed answers: 0

-----------------------------------------------------
Stat for book 4
The ratio betweween generated outputs and inputs is 2.383
Queries judged as coherent 66.8%
Number of failed answers: 4

-----------------------------------------------------
Stat for book 5
The ratio betweween generated outputs and inputs is 3.422
Queries judged as coherent 68.7%
Number of failed answers: 4

-----------------------------------------------------
Stat for book 6
The ratio betweween generated outp